In [2]:
from bs4 import BeautifulSoup
import requests 
from newsdataapi import NewsDataApiClient
import pandas as pd
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine
from collections import Counter
import configparser
%load_ext sql

In [7]:
def read_sql_from_db(table, connection):
    query = pd.read_sql_query(f"""
        SELECT categoryID, wordCount, articleLength, sentiment  
        FROM {table}
        WHERE pubDate BETWEEN CURRENT_DATE - INTERVAL '8 days' AND CURRENT_DATE
    """, connection)

    return query

In [6]:
#create weekly fact table with averageCountAndLength, categoryID, wordCount, articleLentgh and popular sentiments columns
def populate_weekly_fact_table(category, categoryID):
    averageCountAndLength = category.groupby([pd.Grouper(key = 'PubDate', freq = 'W'), 'categoryID']).agg({
        'wordCount': 'mean',
        'articleLength': 'mean'
    }).reset_index()
    
    sentiment_counts = Counter(category['Sentiment'])
    maxCounts = max(sentiment_counts, key = sentiment_counts.get)

    weekly_data = pd.DataFrame ({
        'categoryID':categoryID, 
        'wordCount': averageCountAndLength['wordCount'], 
        'articleLength': averageCountAndLength['articleLength'],
        'popularSentiment': maxCounts
        })

    return weekly_data

In [ ]:
def main():
    try:
        #create a temporary pandas dataframe
        news_weekly_fact_df = pd.DataFrame ()


        config = configparser.ConfigParser()
        config.read('.env')

        #connect postgres db
        db_params = {
            'database': 'news_sentiment_analysis_database',
            'host': 'localhost',
            'user': 'postgres',
            'password' : config['POSTGRES']['password']
                }

        #connect to database
        connection = psycopg2.connect(**db_params)
        cursor = connection.cursor()
        connection.set_session(isolation_level='READ COMMITTED', readonly=False, deferrable=False, autocommit=True)

        #read category tables from news_sentiment_analysis_database 
        business = read_sql_from_db(business, connection)
        crime = read_sql_from_db(crime, connection)
        health = read_sql_from_db(health, connection)
        education = read_sql_from_db(education, connection)
        entertainment = read_sql_from_db(entertainment, connection)
        science = read_sql_from_db(science, connection)

        category_id_mapping = {
                                'business': 101,
                                'crime': 102,
                                'education': 103,
                                'entertainment': 104,
                                'health': 105,
                                'science': 106
                            }
        #add columns to the weekly dataframe
        for category, categoryID in category_id_mapping:
            weekly_data = populate_weekly_fact_table(category, categoryID)
            news_weekly_fact_df = news_weekly_fact_df.append(weekly_data)

    
        #add start datetime and end datatime to the new dataframe
        news_weekly_fact_df['startDate'] = pd.Timestamp.now() - pd.DateOffset(days=7)
        news_weekly_fact_df['endDate'] = pd.Timestamp.now()


        #append the new dataframe to weekly_analysis_database
        try:
            news_weekly_fact_df.to_sql(news_weekly_fact_table, connection, if_exists = 'append', index = False)

            #remove duplicates
            %sql DELETE FROM your_table
            WHERE (article_id, pubDate) IN (
                    SELECT article_id, pubDate
                    FROM (
                            SELECT article_id, pubDate,
                            ROW_NUMBER() OVER (PARTITION BY article_id ORDER BY pubDate) AS row_num
                            FROM news_weekly_fact_table
                        ) AS duplicates
                    WHERE row_num > 1
                    );
            
            print(f'news_weekly_fact_df appended to news_weekly_fact_table sucessfully and duplicates removed')

        except Exception as e:
            print(f'news_weekly_fact_df failed to append to news_weekly_fact_table: {e}')

    
        
    except Exception as e:
        print('An error has occured: {e}')
    finally:
        if connection:
            connection.close()


In [ ]:
if __name__ == "__main__":
    main